# Capture traces for the first three round of NTT(c)

In [1]:
from tqdm.notebook import trange
import numpy as np
import time
import matplotlib.pyplot as plt
import random
import hdf5plugin, h5py
%matplotlib notebook

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_2_1' 

The following code will build the firmware for the target.

In [3]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)

print("INFO: Found ChipWhisperer😍")

prog = cw.programmers.STM32FProgrammer

import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.adc.trig_count                     changed from 162267                    to 12612804                 
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx         

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../firmware/mcu/simpleserial-challenge
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CW308_STM32F4 with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CW308_STM32F4 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
.
.
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
.
-en     simpleserial-challenge.c ...
.
-en     ntt.c ...
-en     packing.c ...
.
-en     polyvec.c ...
-en     poly.c ...
-en     reduce.c ...
Compiling:
-en     rounding.c ...
.
Compiling:
Compiling:
.
.
.
-en     sign.c ...
-en     symmetric-shake.c ...
Compiling:
Compiling:
-en     fips202.c ...
Compiling:
Compiling:
.
.
.
.
.
-en     randombytes.c ...
-en     .././simpleserial/simplese

In file included from .././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition
In file included from .././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal.c:3:
.././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
.
LINKING:
-en     simpleserial-challenge-CW308_STM32F4.elf ...
-e Done!
.
.
.
.
.
Creating load file for Flash: simpleserial-challenge-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-challenge-CW308_STM32F4.elf simpleserial-challenge-CW308_STM32F4.hex
Creating load file for Flash: simpleserial-challenge-CW308_STM32F4.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-challenge-CW308_STM32F4.elf simpleserial-challenge-CW308_STM32F4.bin
Creating load file for EEPROM: simpleserial-challenge-CW308_STM32F4.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-challenge-CW308_STM32F4.elf simpleserial-challenge-CW308_STM32F4.eep ||

In [4]:
cw.program_target(scope, prog, "../../firmware/mcu/simpleserial-challenge/simpleserial-challenge-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 19471 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 19471 bytes


In [5]:
scope.trigger.module = 'basic'
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.clock.clkgen_freq = 25e6 # sample rate = 100M/s
scope.io.hs2 = 'clkgen'
scope.clock.clkgen_src = 'system'
scope.clock.adc_mul = 4
scope.clock.reset_dcms()
reset_target(scope)
scope.adc.offset = 0
# scope.adc.samples = 1118000# 2500000
target.baud *= scope.clock.clkgen_freq / 7.37e6

In [6]:
def encode(slst):
    mask = 2+8*8380417
    buf = []
    for s in slst:
        s += mask
        for i in range(8):
            buf.append(s % 256)
            s = s // 256
    return bytearray(buf)

def decode(buf):
    slst = []
    mask = 2+8*8380417
    buf = list(buf)
    for j in range(len(buf)//8):
        n = 0
        for i in range(8):
            n = n * 256 + buf[j*8+7-i]
        slst.append(n - mask)
    return slst

In [7]:
file_path = "D:/Code_Repo/Redundant_Vulnerabilities_in_Rejection_Sampling/examination/Challenge/DL/"

## first round


In [13]:
with h5py.File(file_path + "fsNTT-profile3k.h5", "r") as f:
    train_label = f["data"][:]
print(train_label.shape)
with h5py.File(file_path + "fsNTT-test1k.h5", "r") as f:
    test_label = f["data"][:]
print(test_label.shape)

(384000, 4)
(128000, 4)


In [14]:
def Trace_caputre_fsntt(nb_trc, label):
    data = np.zeros([nb_trc, scope.adc.samples])
    cnt = 0
    for i in trange(nb_trc, desc = 'fsNTT:\n'):
        scope.arm()
        text = encode([int(label[i][2]), int(label[i][3])])
        target.simpleserial_write(0x1, text)
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 16)
        data[cnt] = np.array(scope.get_last_trace())
        cnt += 1
    return data

In [15]:
scope.adc.samples = 1000

In [17]:
train_data = Trace_caputre_fsntt(train_label.shape[0], train_label)
test_data = Trace_caputre_fsntt(test_label.shape[0], test_label)

fsNTT:
:   0%|          | 0/3000 [00:00<?, ?it/s]

In [31]:
with h5py.File(file_path + "data_fsntt.h5", "w") as f:
    g_train = f.create_group("train")
    g_train.create_dataset("data", data=train_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_train.create_dataset("label", data=train_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test = f.create_group("test")
    g_test.create_dataset("data", data=test_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test.create_dataset("label", data=test_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))

## second round 

In [28]:
with h5py.File(file_path + "secNTT-profile3k.h5", "r") as f:
    train_label = f["data"][:]
print(train_label.shape)
with h5py.File(file_path + "secNTT-test1k.h5", "r") as f:
    test_label = f["data"][:]
print(test_label.shape)

(177246, 8)
(59105, 8)


In [29]:
def Trace_caputre_secntt(nb_trc, label):

    data = np.zeros([nb_trc, scope.adc.samples])
    cnt = 0
    for i in trange(nb_trc, desc = 'secNTT:\n'):
        scope.arm()
        text = encode([label[i][4], label[i][5], label[i][6], label[i][7]])
        target.simpleserial_write(0x02, text)
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 32)
        data[cnt] = np.array(scope.get_last_trace())
        cnt += 1
    return data

In [30]:
scope.adc.samples = 1000

In [31]:
train_data = Trace_caputre_secntt(5000, train_label) # train_label.shape[0]
# test_data = Trace_caputre_fsntt(test_label.shape[0], test_label)

secNTT:
:   0%|          | 0/5000 [00:00<?, ?it/s]

In [11]:
with h5py.File(file_path + "data_secntt.h5", "w") as f:
    g_train = f.create_group("train")
    g_train.create_dataset("data", data=train_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_train.create_dataset("label", data=train_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test = f.create_group("test")
    g_test.create_dataset("data", data=test_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test.create_dataset("label", data=test_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))

## third round

In [44]:
with h5py.File(file_path + "trdNTT-profile3k.h5", "r") as f:
    train_label = f["data"][:]
print(train_label.shape)
with h5py.File(file_path + "trdNTT-test1k.h5", "r") as f:
    test_label = f["data"][:]
print(test_label.shape)

(88650, 12)
(29533, 12)


In [45]:
def Trace_caputre_trdntt(nb_trc, label):

    data = np.zeros([nb_trc, scope.adc.samples])
    cnt = 0
    for i in trange(nb_trc, desc = 'trdNTT:\n'):
        scope.arm()
        text = encode([label[i][4], label[i][5], label[i][6], label[i][7],
                       label[i][8], label[i][9], label[i][10], label[i][11]])
        target.simpleserial_write(0x03, text)
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 64)
        data[cnt] = np.array(scope.get_last_trace())
        cnt += 1
    return data

In [46]:
scope.adc.samples = 1000

In [47]:
train_data = Trace_caputre_trdntt(train_label.shape[0], train_label)
# test_data = Trace_caputre_trdntt(test_label.shape[0], test_label)

trdNTT:
:   0%|          | 0/88650 [00:00<?, ?it/s]

In [48]:
with h5py.File(file_path + "data_trdntt.h5", "w") as f:
    f.create_dataset("data", data=train_data)
    f.create_dataset("label", data=train_label)

In [ ]:
with h5py.File(file_path + "data_trdntt.h5", "w") as f:
    g_train = f.create_group("train")
    g_train.create_dataset("data", data=train_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_train.create_dataset("label", data=train_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test = f.create_group("test")
    g_test.create_dataset("data", data=test_data, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    g_test.create_dataset("label", data=test_label, **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))

## last round

In [8]:
file_path = "D:/Code_Repo/Redundant_Vulnerabilities_in_Rejection_Sampling/examination/Challenge/"

In [9]:
with h5py.File(file_path + "profile_data_challenge.h5", "r") as f:
    profile_data = f["255/data"][:]
print(profile_data.shape)
with h5py.File(file_path + "test_data_challenge.h5", "r") as f:
    test_data = f["data"][:]
print(test_data.shape)
with h5py.File(file_path + "attack_data_challenge.h5", "r") as f:
    attack_data = f["data"][:]
print(attack_data.shape)


(33, 500, 64)
(5000, 64)
(1000, 64)


In [10]:
def Trace_caputre_back_profile(profile_data):
    nb_cls, nb_cls_trc = profile_data.shape[:2]
    trace = np.zeros([(nb_cls)*nb_cls_trc, scope.adc.samples])
    cnt = 0
    for i in trange(nb_cls, desc = 'back:\n'):
        for j in trange(nb_cls_trc, desc = f'HW {i}:\n'):
            scope.arm()
            target.simpleserial_write(0x04, bytearray([int(k) for k in profile_data[i][j]]))
            ret = scope.capture()
            if ret:
                print("Target timed out!")
                continue
            response = target.simpleserial_read('r', 24)
            trace[cnt] = np.array(scope.get_last_trace())
            cnt += 1
    return trace

def Trace_caputre_back_test(test_data):
    nb_trc = test_data.shape[0]
    trace = np.zeros([nb_trc, scope.adc.samples])
    cnt = 0
    for i in trange(nb_trc, desc = 'back:\n'):
        scope.arm()
        target.simpleserial_write(0x04, bytearray([int(k) for k in test_data[i]]))
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 24)
        trace[cnt] = np.array(scope.get_last_trace())
        cnt += 1
    return trace

In [11]:
scope.adc.samples = 30_000 # 1000

In [13]:
wid = 223
offset0 = 160
offset1 = 400

In [ ]:
file_path = "D:/Code_Repo/Redundant_Vulnerabilities_in_Rejection_Sampling/examination/Challenge/"
for q in range(256):
    with h5py.File(file_path + "profile_data_challenge.h5", "r") as f:
        profile_data = f[f"{q}/data"][:]
    print(profile_data.shape)
    profile_trace = Trace_caputre_back_profile(profile_data)    
    test_trace = Trace_caputre_back_test(test_data)
    attack_trace = Trace_caputre_back_test(attack_data)

    with h5py.File(file_path + f"profile_trace_idx_{q}.h5", "w") as f:
        f.create_dataset("trace", data=profile_trace[:, offset0+(q//2)*wid:offset1+(q//2)*wid], **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    with h5py.File(file_path + f"test_trace_idx_{q}.h5", "w") as f:
        f.create_dataset("trace", data=test_trace[:, offset0+(q//2)*wid:offset1+(q//2)*wid], **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    with h5py.File(file_path + f"attack_trace_idx_{q}.h5", "w") as f:
        f.create_dataset("trace", data=attack_trace[:, offset0+(q//2)*wid:offset1+(q//2)*wid], **hdf5plugin.Blosc(cname="zstd", clevel=9, shuffle=hdf5plugin.Blosc.SHUFFLE))
    
    

(33, 500, 64)


back:
:   0%|          | 0/33 [00:00<?, ?it/s]

HW 0:
:   0%|          | 0/500 [00:00<?, ?it/s]

HW 1:
:   0%|          | 0/500 [00:00<?, ?it/s]

HW 2:
:   0%|          | 0/500 [00:00<?, ?it/s]

...

HW 30:
:   0%|          | 0/500 [00:00<?, ?it/s]

HW 31:
:   0%|          | 0/500 [00:00<?, ?it/s]

HW 32:
:   0%|          | 0/500 [00:00<?, ?it/s]

back:
:   0%|          | 0/5000 [00:00<?, ?it/s]

back:
:   0%|          | 0/1000 [00:00<?, ?it/s]